In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [5]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [10]:
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/10/15 13:22:37 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.
2024/10/15 13:22:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/15 13:22:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/159526984123516089/runs/fe22ba5d1f4a46e5b2b65674fcb93c39.
2024/10/15 13:22:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/159526984123516089.
2024/10/15 13:22:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/15 13:22:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/15

### Register the model

In [15]:
model_name = 'RandomForest Anomaly'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 40443a6eaf79498d8fd79f6a0d2d2e2c


Registered model 'RandomForest Anomaly' already exists. Creating a new version of this model...
2024/10/15 16:15:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest Anomaly, version 2
Created version '2' of model 'RandomForest Anomaly'.
2024/10/15 16:15:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/159526984123516089/runs/40443a6eaf79498d8fd79f6a0d2d2e2c.
2024/10/15 16:15:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/159526984123516089.


### LOAD the model

In [17]:
model_version = 2
model_uri = f'models:/{model_name}/{model_version}'
#could also be given as current_model_uri = f"models:/{model_name}@challenger" - based on the alias specified
loaded_model = mlflow.sklearn.load_model(model_uri)

y_pred = loaded_model.predict(X_test)
print(y_pred[:10])

[0 0 0 0 0 0 0 0 0 0]


### Transitioning from Dev Env to Prod Env

In [19]:
dev_model_uri = f'models:/{model_name}/{model_version}'
prod_model = 'anomaly-detection-prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri = dev_model_uri,dst_name = prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '2' of model 'RandomForest Anomaly' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1728989508838, current_stage='None', description='', last_updated_timestamp=1728989508838, name='anomaly-detection-prod', run_id='40443a6eaf79498d8fd79f6a0d2d2e2c', run_link='', source='models:/RandomForest Anomaly/2', status='READY', status_message='', tags={}, user_id='', version='1'>

In [21]:
model_version = 1
prod_model_uri = f'models:/{prod_model}@champion'
#could also be given as current_model_uri = f"models:/{model_name}@challenger" - based on the alias specified
loaded_model = mlflow.sklearn.load_model(prod_model_uri)

y_pred = loaded_model.predict(X_test)
print(y_pred[:10])

[0 0 0 0 0 0 0 0 0 0]
